In [ ]:
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import os 
from selenium.webdriver.common.keys import Keys
from time import sleep
from datetime import datetime

In [ ]:
start_time = datetime.now()

#input file path
filepath = r"C:/workspace/formation/PlusSimple/Scraping/KGS/PlusSimple_ref_2000_4000.xlsx"
sheetname = "to_scrap"
df = pd.read_excel(filepath,sheet_name=sheetname)

#converting Orias codes to 8 digits and into a list
df['orias code'] = [str(x).zfill(8) for x in df['orias code']]
oriascodeList = df['orias code'].tolist()
#creating a blank list to capture unavailable orias codes
notfound=[]

#chrome driver
driver = webdriver.Chrome(r"C:\Users\abachirahltaleb\chrome_for_scraping\chromedriver_win32\chromedriver.exe")

#Output dataframe
data=pd.DataFrame(columns=['Code ORIAS','Siren','Immatriculation','Code NAF','Adresse 1','Adresse 2','Adresse 3'])

#loop through orias codes in input excel
for lst in oriascodeList:    
    try:
        print(lst)
        driver.get("https://www.orias.fr/")
        search = driver.find_element_by_xpath("//input[@id='searchInput']")
        search.clear()
        search.send_keys(lst)
        search.send_keys(Keys.RETURN) 
        sleep(0.2)
        wait = WebDriverWait(driver, 10)

        # orias code variable
        oriascode=lst

        # fetching hyperlinks to intermediaries
        elements = driver.find_elements_by_xpath("//table[@class = 'table table-condensed table-striped table-bordered']//tbody/tr/td[3]//a") 
        if(len(elements)==0):
            notfound.append(lst)
        Hlinks = [elem.get_attribute('href') for elem in elements]
        driver.get(Hlinks[0])
        details=driver.find_element_by_xpath('//*[@class="roundedBox span12"]').text.split("\n")

        #identifying index for siren, codenaf, immatriculation
        siren_loc = [idx for idx, s in enumerate(details) if 'Siren' in s][0]
        codeNAF_loc = [idx for idx, s in enumerate(details) if 'Code NAF' in s][0]
        Immatriculation_loc = [idx for idx, s in enumerate(details) if 'Numéro d\'immatriculation' in s][0]

        # siren, codeNAF, Immatriculation variables
        siren_val = details[siren_loc+1]
        codeNAF_val = details[codeNAF_loc+1]
        Immatriculation_val = details[Immatriculation_loc+1]

        # address variables                                    
        address=driver.find_element_by_xpath('//*[@id="intermediaryCoordinate2"]').text.split("\n")
        if(len(address)== 1): 
            addr1=""
            addr2=""
            addr3=""
        elif(len(address)==2) :   
            addr1=address[1]
            addr2=""
            addr3=""
        elif(len(address)==3) :   
            addr1=address[1]
            addr2=address[2]
            addr3=""
        else:   
            addr1=address[1]
            addr2=address[2]
            addr3=address[3]

        data = data.append({'Code ORIAS':oriascode,'Siren':siren_val,'Immatriculation':Immatriculation_val,
                              'Code NAF':codeNAF_val,'Adresse 1':addr1,'Adresse 2':addr2,'Adresse 3':addr3}, ignore_index=True)


    except:
        continue

driver.close()
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


In [ ]:
data.to_excel(r'C:/workspace/formation/PlusSimple/Scraping/KGS/NotFound_PlusSimple_2000_4000.xlsx')